## Завдання

На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.



Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.



Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

# 1. Встановлення та імпорт бібліотек

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357244 sha256=a2bb1ef3f4d0174771982dd23da836479910659bbc4c5f5e5d8cd4e8b1ee8af7
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
!pip install scikit-surprise

In [3]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD, SVDpp, KNNBaseline, SlopeOne, NMF
from surprise.model_selection import train_test_split, cross_validate


## 2. Завантаження та форматування датасету MovieLens

In [8]:
# Завантаження рейтингових даних MovieLens
#data = Dataset.load_builtin('movielens') #MovieLens # немає такого
# Завантаження датасету Movielens (використовується версія 100k)
data = Dataset.load_builtin('ml-100k')
# URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [9]:
trainset, testset = train_test_split(data, test_size=0.25)

## Крок 3: Побудова моделі та крос-валідація

Модель SVD
SVD (Singular Value Decomposition) - це базова модель матричної факторизації в surprise.

In [10]:
# Побудова моделі SVD та крос-валідація
algo_svd = SVD()

# Крос-валідація
results_svd = cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Виведення результатів
print(results_svd)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9401  0.9315  0.9366  0.9380  0.9367  0.9366  0.0028  
MAE (testset)     0.7409  0.7354  0.7367  0.7417  0.7373  0.7384  0.0025  
Fit time          1.98    1.45    1.50    1.51    2.27    1.74    0.33    
Test time         0.26    0.20    0.12    0.24    0.22    0.21    0.05    
{'test_rmse': array([0.94010594, 0.9315343 , 0.93664785, 0.93803991, 0.93667447]), 'test_mae': array([0.74085452, 0.73535868, 0.73670259, 0.74169695, 0.73730288]), 'fit_time': (1.9821858406066895, 1.452197790145874, 1.4973928928375244, 1.5080924034118652, 2.2653300762176514), 'test_time': (0.2590599060058594, 0.19755077362060547, 0.12323951721191406, 0.2412705421447754, 0.22302865982055664)}


Модель SVD++
SVD++ - розширена версія SVD, яка також враховує взаємодію користувачів і об'єктів (фільмів).

In [11]:
# Побудова моделі SVD++
algo_svdpp = SVDpp()

# Крос-валідація
results_svdpp = cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Виведення результатів
print(results_svdpp)


Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9209  0.9113  0.9198  0.9230  0.9163  0.9182  0.0041  
MAE (testset)     0.7211  0.7166  0.7223  0.7238  0.7166  0.7201  0.0030  
Fit time          28.00   30.70   28.69   28.34   27.92   28.73   1.02    
Test time         5.01    5.24    5.31    5.02    4.77    5.07    0.19    
{'test_rmse': array([0.92085768, 0.9112639 , 0.91975733, 0.9229822 , 0.91626452]), 'test_mae': array([0.72107255, 0.71663853, 0.72226054, 0.72384757, 0.7165833 ]), 'fit_time': (27.99632954597473, 30.69675850868225, 28.68793773651123, 28.342483520507812, 27.917417764663696), 'test_time': (5.005091905593872, 5.238208770751953, 5.310897588729858, 5.021112680435181, 4.768839597702026)}


Модель NMF
NMF (Non-Negative Matrix Factorization) - альтернативний метод матричної факторизації, який обмежує коефіцієнти факторизації до невід'ємних значень.

In [12]:
# Побудова моделі NMF
algo_nmf = NMF()

# Крос-валідація
results_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Виведення результатів
print(results_nmf)


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9560  0.9602  0.9599  0.9667  0.9532  0.9592  0.0046  
MAE (testset)     0.7510  0.7518  0.7555  0.7603  0.7500  0.7537  0.0038  
Fit time          2.15    2.73    2.73    2.36    2.17    2.43    0.26    
Test time         0.11    0.17    0.24    0.12    0.26    0.18    0.06    
{'test_rmse': array([0.95601472, 0.96023466, 0.9598608 , 0.96674807, 0.95320226]), 'test_mae': array([0.75096365, 0.75183934, 0.75550714, 0.76033417, 0.74999349]), 'fit_time': (2.149674415588379, 2.727540969848633, 2.7253494262695312, 2.356273651123047, 2.1685845851898193), 'test_time': (0.10679888725280762, 0.1746511459350586, 0.24307537078857422, 0.12029290199279785, 0.2593834400177002)}


In [13]:
results = {
    "SVD":results_svd,
    "SVDpp":results_svdpp,
    "NMF":results_nmf
}

In [14]:
for model_name, result in results.items():
    mean_rmse = result['test_rmse'].mean()
    mean_mae = result['test_mae'].mean()
    print(f"{model_name} - Mean RMSE: {mean_rmse:.4f}, Mean MAE: {mean_mae:.4f}")

SVD - Mean RMSE: 0.9366, Mean MAE: 0.7384
SVDpp - Mean RMSE: 0.9182, Mean MAE: 0.7201
NMF - Mean RMSE: 0.9592, Mean MAE: 0.7537


Оцінка та вибір моделі


*   RMSE: Це основна метрика, яка оцінює, наскільки передбачення відхиляються від фактичних значень у середньому. Нижчі значення RMSE вказують на кращу модель.
*   MAE: Це середня абсолютна похибка, яка також використовується для оцінки точності передбачень. Як і RMSE, нижчі значення MAE вказують на кращу модель.

Однак, якщо одна модель має значно нижчий RMSE, але дещо вищий MAE (або навпаки), вам доведеться зважити, яка метрика є більш важливою для вашого конкретного випадку.




## Згідно цих критеріїв та отриманих значень моделі, для наших даних найкраща модель - це SVDpp, яка має найменші значення Mean RMSE: 0.9182, Mean MAE: 0.7201